# Employee Productivity Prediction Using Tree Models

## Load Data

In [121]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/garments_worker_productivity.csv', parse_dates=['date'])

## Examine

In [122]:
df.head()

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,2015-01-01,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,2015-01-01,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,2015-01-01,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,2015-01-01,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,2015-01-01,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   1197 non-null   datetime64[ns]
 1   quarter                1197 non-null   object        
 2   department             1197 non-null   object        
 3   day                    1197 non-null   object        
 4   team                   1197 non-null   int64         
 5   targeted_productivity  1197 non-null   float64       
 6   smv                    1197 non-null   float64       
 7   wip                    691 non-null    float64       
 8   over_time              1197 non-null   int64         
 9   incentive              1197 non-null   int64         
 10  idle_time              1197 non-null   float64       
 11  idle_men               1197 non-null   int64         
 12  no_of_style_change     1197 non-null   int64         
 13  no_

### Column Descriptions

date: date in MM-DD-YYYY

quarter: a portion of the month — month was divided into four quarters

department: associated department with the instance

day: day of the week

team: associated team number with the instance

targeted_productivity: targeted productivity set by the authority for each team for each day

smv: standard minute value — the allocated time for a task

wip: work in progress — includes the number of unfinished items for products

over_time: represents the amount of overtime by each team in minutes

incentive: represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action

idle_time: the duration of time when the production was interrupted due to several reasons

idle_men: the number of workers who were idle due to production interruption

no_of_style_change: number of changes in the style of a particular product

no_of_workers: number of workers on each team

actual_productivity: the actual % of productivity that was delivered by the workers — it ranges from 0 to 1.

In [124]:
df.shape

(1197, 15)

### Check Nulls

In [125]:
df.isnull().sum()

date                       0
quarter                    0
department                 0
day                        0
team                       0
targeted_productivity      0
smv                        0
wip                      506
over_time                  0
incentive                  0
idle_time                  0
idle_men                   0
no_of_style_change         0
no_of_workers              0
actual_productivity        0
dtype: int64

### Check Categorical Column Value Counts

In [126]:
cols = ['quarter', 'department', 'day', 'team']

for col in cols:
    print(f'{df[col].value_counts()}\n')

quarter
Quarter1    360
Quarter2    335
Quarter4    248
Quarter3    210
Quarter5     44
Name: count, dtype: int64

department
sweing        691
finishing     257
finishing     249
Name: count, dtype: int64

day
Wednesday    208
Sunday       203
Tuesday      201
Thursday     199
Monday       199
Saturday     187
Name: count, dtype: int64

team
8     109
2     109
4     105
1     105
9     104
10    100
12     99
7      96
3      95
6      94
5      93
11     88
Name: count, dtype: int64



### Quick Findings:

#### `quarter`:

- 5th quarter exists.

#### `department`:

- Typo in "sweing"
- Two "finishing" values, perhaps whitespace

#### `day`:

- All days except "Friday", perhaps no production on Fridays

#### `team`:

- 12 total teams

### Comparing Related Columns

In [127]:
df[['targeted_productivity', 'actual_productivity']].describe()

,targeted_productivity,actual_productivity
count,1197.000000,1197.000000
mean,0.729632,0.735091
std,0.097891,0.174488
min,0.070000,0.233705
25%,0.700000,0.650307
50%,0.750000,0.773333
75%,0.800000,0.850253
max,0.800000,1.120437


We can see some abnormal values, specifically within `actual_productivity`. The max value seems to be above the possible productivity value (a scale from 0-100%), as it is observed as 1.12 (112%) productivity. Let's check this column's unique values.

In [128]:
df.loc[df['actual_productivity'] > 1, 'actual_productivity'].value_counts(dropna=False).sort_index(ascending=False)

actual_productivity
1.120437     1
1.108125     1
1.100484     1
1.096633     1
1.059621     1
1.057963     1
1.050667     1
1.050281     1
1.033570     1
1.033156     1
1.020000     1
1.011562     1
1.004889     1
1.001417     1
1.000671     1
1.000602     1
1.000457     1
1.000446     1
1.000402     1
1.000345     2
1.000230    11
1.000066     1
1.000019     4
Name: count, dtype: int64

In [129]:
print('Observations Where Actual Productivity > 1: ',df.loc[df['actual_productivity'] > 1, 'actual_productivity'].count())
print('Total Observations ', df.shape[0])
print('Proportion: ', round(df.loc[df['actual_productivity'] > 1, 'actual_productivity'].count() / df.shape[0], 2))

Observations Where Actual Productivity > 1:  37
Total Observations  1197
Proportion:  0.03


We could deal with this problem a couple of ways. 

We could: 

- Scale the data
- Replace the values with 1.0 as they effectively max out the scale

As these abnormalities seem to only occur on a small fraction of our overall observations, we will simply replace the values with the maximum of our scale, in order to avoid altering the scaling between `target_productivity` as well. We could have also scaled `target_productivity` to the same scaling, but this method is quicker as there are only a few problematic observations.

We'll clean this in the next section.

## Cleaning

### `actual_productivity`

We'll simply cap off our abnormal values to max out at 1.0.

In [130]:
df.loc[df['actual_productivity'] > 1, 'actual_productivity'] = 1

In [131]:
df[['targeted_productivity', 'actual_productivity']].describe()

,targeted_productivity,actual_productivity
count,1197.000000,1197.000000
mean,0.729632,0.734461
std,0.097891,0.173380
min,0.070000,0.233705
25%,0.700000,0.650307
50%,0.750000,0.773333
75%,0.800000,0.850253
max,0.800000,1.000000


### `wip` and `department` columns

In [132]:
df['wip'].value_counts(dropna=False).sort_values(ascending=False)

wip
NaN       506
1039.0      5
1282.0      4
1108.0      3
1138.0      3
         ... 
1674.0      1
290.0       1
971.0       1
992.0       1
984.0       1
Name: count, Length: 549, dtype: int64

In [133]:
df[df['wip'].isnull()].sample(5)

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
57,2015-01-04,Quarter1,finishing,Sunday,8,0.80,2.90,NaN,960,0,0.0,0,0,8.0,0.682708
18,2015-01-01,Quarter1,finishing,Thursday,11,0.70,4.15,NaN,1440,0,0.0,0,0,12.0,0.436326
524,2015-01-31,Quarter5,finishing,Saturday,1,0.75,3.94,NaN,1800,0,0.0,0,0,15.0,0.971867
300,2015-01-18,Quarter3,finishing,Sunday,11,0.65,4.15,NaN,2160,0,0.0,0,0,12.0,0.838384
944,2015-02-26,Quarter4,finishing,Thursday,5,0.80,4.60,NaN,7560,0,0.0,0,0,18.0,0.921605


We will drill down into cases surrounding null values. Perhaps null `wip` entries are due to preceding or upcoming weeks.

In [134]:
df.iloc[309:322]

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
309,2015-01-18,Quarter3,sweing,Sunday,12,0.80,26.87,585.0,5670,56,0.0,0,0,31.5,0.662256
310,2015-01-18,Quarter3,sweing,Sunday,3,0.65,22.52,1138.0,10170,35,0.0,0,0,56.5,0.650417
311,2015-01-18,Quarter3,sweing,Sunday,11,0.65,14.89,1140.0,10260,44,0.0,0,0,57.0,0.650300
312,2015-01-18,Quarter3,sweing,Sunday,2,0.65,22.94,1025.0,10170,26,0.0,0,0,56.5,0.650243
313,2015-01-18,Quarter3,finishing,Sunday,8,0.80,2.90,NaN,1440,0,0.0,0,0,8.0,0.639867
314,2015-01-18,Quarter3,sweing,Sunday,5,0.60,20.40,1031.0,10440,30,0.0,0,0,58.0,0.600063
315,2015-01-18,Quarter3,finishing,Sunday,9,0.80,3.94,NaN,1800,0,0.0,0,0,10.0,0.582045
316,2015-01-18,Quarter3,sweing,Sunday,1,0.80,49.10,1381.0,10350,24,0.0,0,0,57.5,0.403242
317,2015-01-18,Quarter3,finishing,Sunday,10,0.70,3.94,NaN,1440,0,0.0,0,0,8.0,0.235795
318,2015-01-19,Quarter3,finishing,Monday,4,0.70,4.30,NaN,5040,0,0.0,0,0,28.0,0.977273


Seeing this specific slice of our data, we can identify an interesting pattern. The cases in which `wip` is missing all occur in the finishing department. Lets see what other entries the finishing department in particular contains in comparison to the rest. Also we need to clean `department` anyways, due to two existing "finishing" values.

In [135]:
df['department'].unique().tolist()

['sweing', 'finishing ', 'finishing']

We can see that our earlier theory was correct—there is whitespace proceeding one of the values. We will correct this.

In [136]:
df['department'] = df['department'].replace('finishing ', 'finishing')

We will also correct the type on "sewing"

In [137]:
df['department'] = df['department'].replace('sweing', 'sewing')

In [138]:
df['department'].unique().tolist()

['sewing', 'finishing']

Now we can check for null `wip` values by `department`

In [139]:
df.loc[df['department'] == 'sewing', 'wip'].isnull().sum()

np.int64(0)

In [140]:
df.loc[df['department'] == 'finishing', 'wip'].isnull().sum()

np.int64(506)

Interesting. Let's see the total amount of entries for the `finishing` department.

In [141]:
df.loc[df['department'] == 'finishing', 'wip'].shape

(506,)

We can now see that every entry for the `wip` column within the `finishing` department is missing.

This is most likely due to `finishing` leaving no works-in-progress, so we can effectively set these instances to 0.

In [142]:
df['wip'] = df['wip'].fillna(0)

In [143]:
df.isnull().sum()

date                     0
quarter                  0
department               0
day                      0
team                     0
targeted_productivity    0
smv                      0
wip                      0
over_time                0
incentive                0
idle_time                0
idle_men                 0
no_of_style_change       0
no_of_workers            0
actual_productivity      0
dtype: int64

### `quarter` column

For this column we will simply alter the "Quarter5" entries to "Quarter4".

In [144]:
df['quarter'] = df['quarter'].replace('Quarter5', 'Quarter4')

In [145]:
df['quarter'].value_counts()

quarter
Quarter1    360
Quarter2    335
Quarter4    292
Quarter3    210
Name: count, dtype: int64

## Re-examine After Cleaning

In [146]:
df.columns

Index(['date', 'quarter', 'department', 'day', 'team', 'targeted_productivity',
       'smv', 'wip', 'over_time', 'incentive', 'idle_time', 'idle_men',
       'no_of_style_change', 'no_of_workers', 'actual_productivity'],
      dtype='object')

In [147]:
df.head()

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,2015-01-01,Quarter1,sewing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,2015-01-01,Quarter1,finishing,Thursday,1,0.75,3.94,0.0,960,0,0.0,0,0,8.0,0.886500
2,2015-01-01,Quarter1,sewing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,2015-01-01,Quarter1,sewing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,2015-01-01,Quarter1,sewing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [148]:
df.describe(include='all')

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
count,1197,1197,1197,1197,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000
unique,NaN,4,2,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Quarter1,sewing,Wednesday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,360,691,208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2015-02-04 10:56:50.526315776,NaN,NaN,NaN,6.426901,0.729632,15.062172,687.228070,4567.460317,38.210526,0.730159,0.369256,0.150376,34.609858,0.734461
min,2015-01-01 00:00:00,NaN,NaN,NaN,1.000000,0.070000,2.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.233705
25%,2015-01-18 00:00:00,NaN,NaN,NaN,3.000000,0.700000,3.940000,0.000000,1440.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.650307
50%,2015-02-03 00:00:00,NaN,NaN,NaN,6.000000,0.750000,15.260000,586.000000,3960.000000,0.000000,0.000000,0.000000,0.000000,34.000000,0.773333
75%,2015-02-23 00:00:00,NaN,NaN,NaN,9.000000,0.800000,24.260000,1083.000000,6960.000000,50.000000,0.000000,0.000000,0.000000,57.000000,0.850253
max,2015-03-11 00:00:00,NaN,NaN,NaN,12.000000,0.800000,54.560000,23122.000000,25920.000000,3600.000000,300.000000,45.000000,2.000000,89.000000,1.000000


We will drop the `date` column as this is effectively dealt with using `quarter` and `day`. Also, we can see that our data spans only 3 months, so the `date` column does not really offer much extra data.

We *could* feature-engineer a `month` column. However, fitting with only 3 months of data could potentially negatively effect our results, especially if we wanted to predict on any other months.

### Drop `date`

In [149]:
df.drop(columns=['date'], inplace=True)

## Prepare Column Types (`quarter`, `day`)

We need to convert these columns to integer types in order to build our model.

### Check Current Unique Values

In [150]:
df['quarter'].unique()

array(['Quarter1', 'Quarter2', 'Quarter3', 'Quarter4'], dtype=object)

In [151]:
df['day'].unique()

array(['Thursday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday'],
      dtype=object)

### Map to New Values

In [152]:
df['quarter'] = df['quarter'].map({'Quarter1': 1, 'Quarter2': 2, 'Quarter3': 3, 'Quarter4': 4})

df['day'] = df['day'].map({'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6, 'Saturday': 7})

In [153]:
df['quarter'].unique()

array([1, 2, 3, 4])

In [154]:
df['day'].unique()

array([5, 7, 1, 2, 3, 4])

In [155]:
df.dtypes

quarter                    int64
department                object
day                        int64
team                       int64
targeted_productivity    float64
smv                      float64
wip                      float64
over_time                  int64
incentive                  int64
idle_time                float64
idle_men                   int64
no_of_style_change         int64
no_of_workers            float64
actual_productivity      float64
dtype: object

Great! Now we just need to encode `department`.

## Encoding `department`